In [1]:
# import libraries
import pandas as pd
import numpy as np
from sodapy import Socrata
from google.cloud import bigquery
from google.oauth2 import service_account

# Building an ETL Pipeline

### Setup your NYC Open Data variables ( app token change needed) 

In [2]:
data_url = 'data.cityofnewyork.us'
data_set = 'ipu4-2q9a'
app_token = ''

In [3]:
nyc_open_data_client = Socrata(data_url, app_token, timeout = 200)
print(f"nyc open data client name is: {nyc_open_data_client}")
print(f"nyc open data client data type is: {type(nyc_open_data_client)}")

nyc open data client name is: <sodapy.socrata.Socrata object at 0x000002282752A220>
nyc open data client data type is: <class 'sodapy.socrata.Socrata'>


### Step 2: Setup your Google BigQuery variables (Path & dataset_id change needed)

In [4]:
key_path =  r''

In [5]:
# run this cell without changing anything to setup your credentials
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
bigquery_client = bigquery.Client(credentials = credentials,
                                 project = credentials.project_id)

print(f"bigquery client name is: {bigquery_client}")
print(f"bigquery client data type is: {type(bigquery_client)}")

bigquery client name is: <google.cloud.bigquery.client.Client object at 0x000002282A6659D0>
bigquery client data type is: <class 'google.cloud.bigquery.client.Client'>


In [6]:
dataset_id = ''   # PASTE THIS DATASET ID FROM ABOVE STEPS

dataset_id = dataset_id.replace(':', '.')
print(f"your dataset_id is: {dataset_id}")

your dataset_id is: cis-9440-361019.project


### Step 3: Extract data

#### The fist dataset (DOB Permit Issuance)

In [7]:
# Get the total number of records in our the entire data set
total_record_count = nyc_open_data_client.get(data_set, select = "COUNT(*)")
print(f"total records in {data_set}: {total_record_count[0]['COUNT']}")

total records in ipu4-2q9a: 3922561


In [8]:
target_record_count = nyc_open_data_client.get(data_set,
                                              where = "permit_type='NB'",
                                              select= "COUNT(*)")


print(f"target records in {data_set}: {int(target_record_count[0]['COUNT'])}")

target records in ipu4-2q9a: 258586


In [9]:
# Now, loop through target data set to pull all rows in chunks (we cannot pull all rows at once)
# AGAIN, UPDATE WHERE FILTER INSIDE BELOW FUNCTION

def extract_socrata_data(chunk_size = 2500,
                         data_set = data_set,
                         where =None):
    
    # measure time this function takes
    import time
    start_time = time.time()
    
    # get total number or records
    if where == None:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     select= "COUNT(*)")[0]["COUNT"])
    else:
        total_records = int(nyc_open_data_client.get(data_set,
                                                     where = where,
                                                     select= "COUNT(*)")[0]["COUNT"])
    
    # start at 0, empty list for results
    start = 0                   
    results = []                

    while True:

        if where == None:
            # fetch the set of records starting at 'start'
            results.extend(nyc_open_data_client.get(data_set,
                                                    offset = start,
                                                    limit = chunk_size))
            
        elif where != None:
            results.extend(nyc_open_data_client.get(data_set,
                                                    where = where,
                                                    offset = start,
                                                    limit = chunk_size))
        # update the starting record number
        start = start + chunk_size

        # if we have fetched all of the records (we have reached total_records), exit loop
        if (start > total_records):
            break

    # convert the list into a pandas data frame
    data = pd.DataFrame.from_records(results)

    end_time = time.time()
    print(f"function took {round(end_time - start_time, 1)} seconds")

    print(f"the shape of your dataframe is: {data.shape}")
    return data

In [10]:
data = extract_socrata_data(chunk_size = 2500,
                            data_set = data_set,
                            where = "permit_type='NB'")

function took 72.4 seconds
the shape of your dataframe is: (258586, 58)


In [11]:
data.columns

Index(['borough', 'bin__', 'house__', 'street_name', 'job__', 'job_doc___',
       'job_type', 'self_cert', 'block', 'lot', 'community_board', 'zip_code',
       'bldg_type', 'residential', 'permit_status', 'filing_status',
       'permit_type', 'permit_sequence__', 'site_fill', 'filing_date',
       'issuance_date', 'expiration_date', 'job_start_date',
       'permittee_s_first_name', 'permittee_s_last_name',
       'permittee_s_business_name', 'permittee_s_phone__',
       'permittee_s_license_type', 'permittee_s_license__',
       'superintendent_first___last_name', 'superintendent_business_name',
       'owner_s_business_type', 'non_profit', 'owner_s_business_name',
       'owner_s_first_name', 'owner_s_last_name', 'owner_s_phone__',
       'dobrundate', 'permit_si_no', 'gis_latitude', 'gis_longitude',
       'gis_council_district', 'gis_census_tract', 'gis_nta_name',
       'site_safety_mgr_s_first_name', 'site_safety_mgr_s_last_name',
       'site_safety_mgr_business_name', 'spec

In [12]:
data = data[["borough","street_name","zip_code","bldg_type","permit_type","permit_status","permittee_s_business_name","filing_date","gis_latitude","gis_longitude"]]

In [13]:
data

,borough,street_name,zip_code,bldg_type,permit_type,permit_status,permittee_s_business_name,filing_date,gis_latitude,gis_longitude
0,BROOKLYN,HERKIMER STREET,11213,2,NB,ISSUED,AB PLUS R CONSTRUCTION IN,05/10/2022,40.678919,-73.936482
1,BRONX,WHITE PLAINS ROAD,10470,2,NB,ISSUED,"SKF GENERAL CONTRACTORS,",05/10/2022,40.898971,-73.853874
2,BROOKLYN,WILLOUGHBY AVENUE,11205,2,NB,ISSUED,MC GOWAN BUILDERS INC,05/10/2022,40.692368,-73.961350
3,BROOKLYN,NEW YORK AVENUE,11203,2,NB,ISSUED,"M & Y DEVELOPERS, INC",05/10/2022,40.641580,-73.945684
4,BRONX,VYSE AVE,10460,2,NB,ISSUED,KIRI CONSTRUCTION CORP,05/10/2022,40.842322,-73.881900
...,...,...,...,...,...,...,...,...,...,...
258581,BROOKLYN,HURON STREET,11222,2,NB,ISSUED,QUAD CONSTRUCTION NY,11/25/2022,40.733089,-73.954219
258582,BROOKLYN,LIBERTY AVE,11208,2,NB,ISSUED,"URBAN ECOSPACES, INC",11/25/2022,40.677058,-73.878443
258583,BROOKLYN,PACIFIC STREET,11238,2,NB,ISSUED,FAZIO CONSTRUCTION GROUP,11/25/2022,40.680586,-73.965943
258584,BROOKLYN,EAST 53 STREET,11203,2,NB,ISSUED,CITYSCAPE BUILDERS LLC,11/25/2022,40.658937,-73.928211


In [14]:
data['filing_date']= pd.to_datetime(data['filing_date'])

In [15]:
data=data[(data['filing_date'] >= '2019-01-01') & (data['filing_date'] <= '2021-12-31')]

In [16]:
data.isnull().sum()

borough                        0
street_name                    0
zip_code                      40
bldg_type                      0
permit_type                    0
permit_status                 10
permittee_s_business_name     73
filing_date                    0
gis_latitude                 458
gis_longitude                458
dtype: int64

In [17]:
data=data.dropna()

In [18]:
data.isnull().sum()

borough                      0
street_name                  0
zip_code                     0
bldg_type                    0
permit_type                  0
permit_status                0
permittee_s_business_name    0
filing_date                  0
gis_latitude                 0
gis_longitude                0
dtype: int64

In [19]:
data = data.drop_duplicates()

In [20]:
data=data.reset_index(drop=True)

In [21]:
data['year'] = pd.DatetimeIndex(data['filing_date']).year

In [22]:
data['zip_code'] = data['zip_code'].astype('int64')
data['gis_latitude'] = data['gis_latitude'].astype('float')
data['gis_longitude'] = data['gis_longitude'].astype('float')

In [23]:
data

,borough,street_name,zip_code,bldg_type,permit_type,permit_status,permittee_s_business_name,filing_date,gis_latitude,gis_longitude,year
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019
1,MANHATTAN,WEST 105TH STREET,10025,1,NB,ISSUED,TOP ROCK DEVELOPMENT AND,2021-11-17,40.798286,-73.962716,2021
2,BROOKLYN,61 STREET,11219,1,NB,ISSUED,KAUFMAN GROUP INC,2020-06-24,40.626170,-73.995670,2020
3,BRONX,SAINT LAWRENCE AVENUE,10473,1,NB,ISSUED,RASHBI DEVELOPMENT INC,2021-04-06,40.813572,-73.863039,2021
4,BROOKLYN,BEACH 38 STREET,11224,1,NB,ISSUED,SAPOL CONSTRUCTION LLC,2021-04-20,40.573632,-74.004341,2021
...,...,...,...,...,...,...,...,...,...,...,...
19228,BROOKLYN,SOUTH 5 STREET,11211,2,NB,ISSUED,TRIBOROUGH CONSTRUCTION S,2019-05-30,40.707104,-73.951759,2019
19229,BROOKLYN,SOUTH 5 STREET,11211,2,NB,ISSUED,DLC DEVELOPMENT CORP,2020-05-08,40.707104,-73.951759,2020
19230,QUEENS,103 AVE,11419,1,NB,ISSUED,CUSTOMBUILT HOMES INC,2020-08-17,40.688931,-73.821593,2020
19231,QUEENS,103 AVE,11419,1,NB,ISSUED,CUSTOMBUILT HOMES INC,2021-05-21,40.688931,-73.821593,2021


#### The second dataset from Annualized Sales Update

In [24]:
data1 = pd.read_csv('Borough_Sales_History_Merged.csv')

In [25]:
data1 = data1.drop_duplicates()

In [26]:
data1.columns

Index(['Unnamed: 0', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AS OF FINAL ROLL 18/19', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AS OF FINAL ROLL 18/19', 'ADDRESS', 'APARTMENT NUMBER',
       'ZIP CODE', 'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [27]:
data1.drop(['Unnamed: 0', 'NEIGHBORHOOD','BOROUGH', 'BUILDING CLASS CATEGORY','TAX CLASS AS OF FINAL ROLL 18/19','BLOCK','LOT','EASE-MENT','BUILDING CLASS AS OF FINAL ROLL 18/19','APARTMENT NUMBER','RESIDENTIAL UNITS','COMMERCIAL UNITS','TOTAL UNITS','LAND SQUARE FEET','GROSS SQUARE FEET','YEAR BUILT',"TAX CLASS AT TIME OF SALE",'BUILDING CLASS AT TIME OF SALE','ADDRESS'], axis=1, inplace=True)

In [28]:
data1=data1.dropna()

In [29]:
data1['ZIP CODE'] = data1['ZIP CODE'].astype('int64')
data1['SALE PRICE'] = data1['SALE PRICE'].astype('int64')
data1['SALE DATE']= pd.to_datetime(data1['SALE DATE'])

In [30]:
data1.rename(columns = {'ZIP CODE':'zip_code', 'SALE PRICE':'median_housing_price',
                              'SALE DATE':'sale_date'}, inplace = True)

In [31]:
data1['year'] = pd.DatetimeIndex(data1['sale_date']).year
data1=data1[data1['median_housing_price']>100]
data1.head()

,zip_code,median_housing_price,sale_date,year
0,10009,3200000,2019-07-24,2019
3,10009,6300000,2019-04-30,2019
5,10009,1950000,2019-08-08,2019
6,10009,14000000,2019-01-09,2019
8,10009,600000,2019-12-12,2019


In [32]:
data1.dtypes

zip_code                         int64
median_housing_price             int64
sale_date               datetime64[ns]
year                             int64
dtype: object

In [33]:
data1.to_csv("price.csv",index=False)

In [34]:
data1=data1.groupby(['zip_code', 'year']).median()
data1

median_housing_price
zip_code year                      
10001    2019             2938784.0
         2020             2762500.0
         2021             2045000.0
10002    2019             1419368.0
         2020             1083750.0
...                             ...
11694    2020              624391.0
         2021              722500.0
11697    2019              433500.0
         2020              500000.0
         2021              625000.0

[559 rows x 1 columns]

#### Merge two datasets together

In [35]:
data1.to_csv("price.csv",index=False)

In [36]:
data= data.merge(data1, on=["zip_code","year"])

In [37]:
data['new_permit_quantity']=1

In [38]:
data.rename(columns = {'street_name':'street','permittee_s_business_name':'permittees_business_name',
                              'filing_date':'date','gis_latitude':'latitude','gis_longitude':'longitude'}, inplace = True)

In [39]:
data['median_housing_price'] = data['median_housing_price'].astype('int64')

In [40]:
data

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1
2,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-03-28,40.636955,-73.950995,2019,772000,1
3,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1
4,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19217,MANHATTAN,EAST 132 STREET,10037,2,NB,ISSUED,DEDAJ CONSTRUCTION CORP,2021-06-16,40.810646,-73.938541,2021,450000,1
19218,MANHATTAN,BROADWAY,10034,2,NB,ISSUED,MEGA CONTRACTING GP LLC,2021-08-03,40.865947,-73.926075,2021,465000,1
19219,MANHATTAN,WEST 218 STREET,10034,2,NB,ISSUED,"BOLIVAR BUILDERS, LLC",2021-09-30,40.870513,-73.912431,2021,465000,1
19220,MANHATTAN,BROADWAY,10034,2,NB,ISSUED,MEGA CONTRACTING GP LLC,2021-10-12,40.865947,-73.926075,2021,465000,1


In [41]:
#We found two error data.zip code 10312 should belong to STATEN ISLAND NOT BK

In [42]:
data.loc[(data['zip_code'] ==10312) & (data['borough']=='BROOKLYN')]

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity
1967,BROOKLYN,LYNCH STREET,10312,1,NB,ISSUED,CORNERSTONE BUILDERS NY L,2019-07-12,40.701689,-73.954633,2019,568000,1
3152,BROOKLYN,LYNCH STREET,10312,1,NB,ISSUED,CORNERSTONE BUILDERS NY L,2020-07-08,40.701689,-73.954633,2020,550000,1


In [43]:
data.loc[data.zip_code == 10312, 'borough'] = 'STATEN ISLAND'

In [44]:
data.to_csv("data.csv",index=False)

### Step 4: Data Profiling

In [45]:
data.columns

Index(['borough', 'street', 'zip_code', 'bldg_type', 'permit_type',
       'permit_status', 'permittees_business_name', 'date', 'latitude',
       'longitude', 'year', 'median_housing_price', 'new_permit_quantity'],
      dtype='object')

In [46]:
# create and run a function to ceate data profiling dataframe

def create_data_profiling_df(data):
    
    # create an empty dataframe to gather information about each column
    data_profiling_df = pd.DataFrame(columns = ["column_name",
                                                "column_type",
                                                "unique_values",
                                                "duplicate_values",
                                                "null_values",
                                                "non_null_values"])

    # loop through each column to add rows to the data_profiling_df dataframe
    for column in data.columns:

        info_dict = {}

        try:
            info_dict["column_name"] = column
            info_dict["column_type"] = data[column].dtypes
            info_dict["unique_values"] = len(data[column].unique())
            info_dict["duplicate_values"] = data[column].count() - len(data[column].dropna().unique())
            info_dict["null_values"] = data[column].isna().sum()
            info_dict["non_null_values"] = data[column].count()

        except:
            print(f"unable to read column: {column}, you may want to drop this column")

        data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)

    data_profiling_df.sort_values(by = ['unique_values', "non_null_values"],
                                  ascending = [False, False],
                                  inplace=True)
    
    return data_profiling_df

In [47]:
data_profiling_df = create_data_profiling_df(data=data)

C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_17828\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_17828\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_17828\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_profiling_df = data_profiling_df.append(info_dict, ignore_index=True)
C:\Users\XIAOJIN LI\AppData\Local\Temp\ipykernel_17828\3975332968.py:29: FutureWarning: The frame.append method is deprecated and will be removed from 

In [48]:
data_profiling_df

,column_name,column_type,unique_values,duplicate_values,null_values,non_null_values
9,longitude,float64,6962,12260,0,19222
8,latitude,float64,6945,12277,0,19222
1,street,object,3187,16035,0,19222
6,permittees_business_name,object,1789,17433,0,19222
7,date,datetime64[ns],886,18336,0,19222
11,median_housing_price,int64,379,18843,0,19222
2,zip_code,int64,178,19044,0,19222
0,borough,object,5,19217,0,19222
5,permit_status,object,3,19219,0,19222
10,year,int64,3,19219,0,19222


### Step 5: Data Cleansing

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19222 entries, 0 to 19221
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   borough                   19222 non-null  object        
 1   street                    19222 non-null  object        
 2   zip_code                  19222 non-null  int64         
 3   bldg_type                 19222 non-null  object        
 4   permit_type               19222 non-null  object        
 5   permit_status             19222 non-null  object        
 6   permittees_business_name  19222 non-null  object        
 7   date                      19222 non-null  datetime64[ns]
 8   latitude                  19222 non-null  float64       
 9   longitude                 19222 non-null  float64       
 10  year                      19222 non-null  int64         
 11  median_housing_price      19222 non-null  int64         
 12  new_permit_quantit

In [50]:
# find number of duplicate rows

print(f"number of duplicate rows: {len(data[data.duplicated()])}")

number of duplicate rows: 0


In [51]:
data = data.drop_duplicates(keep = 'first')
print(f"number of rows after duplicates dropped: {len(data)}")

number of rows after duplicates dropped: 19222


### Step 6: Create location dimension table

In [52]:
# first, copy the entire table
Location_dim = data.copy()

In [53]:
# second, subset for only the wanted columns in the dimension
Location_dim = Location_dim[['borough','zip_code','street','longitude','latitude']]
Location_dim.shape

(19222, 5)

In [54]:
# third, drop duplicate rows in dimension
unique_row =['borough','zip_code','street','longitude','latitude']
Location_dim = Location_dim.drop_duplicates(subset = unique_row, keep = 'first')
Location_dim = Location_dim.reset_index(drop = True)
Location_dim.shape

(7218, 5)

In [55]:
# fourth, add location_id as a surrogate key
Location_dim.insert(0, 'location_id', range(1, 1 + len(Location_dim)))

In [56]:
Location_dim

,location_id,borough,zip_code,street,longitude,latitude
0,1,BROOKLYN,11210,EAST 22 STREET,-73.953736,40.622307
1,2,BROOKLYN,11210,FLATBUSH AVENUE,-73.953055,40.637919
2,3,BROOKLYN,11210,ROGERS AVENUE,-73.950995,40.636955
3,4,BROOKLYN,11210,EAST 32 STREET,-73.945784,40.633264
4,5,BROOKLYN,11210,NEW YORK AVENUE,-73.945176,40.636835
...,...,...,...,...,...,...
7213,7214,MANHATTAN,10014,GREENWICH STREET,-74.008757,40.727941
7214,7215,MANHATTAN,10038,BEEKMAN STREET,-74.006298,40.711050
7215,7216,MANHATTAN,10034,BROADWAY,-73.926075,40.865947
7216,7217,MANHATTAN,10034,WEST 218 STREET,-73.912431,40.870513


In [57]:
# fifth, add the location_id to the df table
data = data.merge(Location_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

In [58]:
data.head()

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity,location_id
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1,1
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1,2
2,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-03-28,40.636955,-73.950995,2019,772000,1,3
3,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1,4
4,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1,5


### Step 7: Create date dimension table

In [59]:
# first, copy the entire table
dateDim = data.copy()

In [60]:
dateDim = dateDim[["date",'year']]

In [61]:
dateDim=dateDim[~dateDim['date'].dt.date.duplicated()]
dateDim['month'] = dateDim['date'].dt.month
# create date_id column in the dateDim table
dateDim['date_id'] = dateDim['date'].apply(lambda x: pd.to_datetime(x).strftime("%Y%m%d"))
dateDim = dateDim.reset_index(drop = True)

In [62]:
# create date_id column in the data table
data['date_id'] = data['date'].apply(lambda x: pd.to_datetime(x).strftime("%Y%m%d"))

In [63]:
data

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity,location_id,date_id
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1,1,20190924
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1,2,20190510
2,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-03-28,40.636955,-73.950995,2019,772000,1,3,20190328
3,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1,4,20190430
4,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1,5,20190103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19217,MANHATTAN,EAST 132 STREET,10037,2,NB,ISSUED,DEDAJ CONSTRUCTION CORP,2021-06-16,40.810646,-73.938541,2021,450000,1,4741,20210616
19218,MANHATTAN,BROADWAY,10034,2,NB,ISSUED,MEGA CONTRACTING GP LLC,2021-08-03,40.865947,-73.926075,2021,465000,1,7216,20210803
19219,MANHATTAN,WEST 218 STREET,10034,2,NB,ISSUED,"BOLIVAR BUILDERS, LLC",2021-09-30,40.870513,-73.912431,2021,465000,1,7217,20210930
19220,MANHATTAN,BROADWAY,10034,2,NB,ISSUED,MEGA CONTRACTING GP LLC,2021-10-12,40.865947,-73.926075,2021,465000,1,7216,20211012


### Step 8: Create permit dimension table

In [64]:
# first, copy the entire table

Permit_dim = data.copy()

In [65]:
# second, subset for only the wanted columns in the dimension

Permit_dim = Permit_dim[["permit_type","permittees_business_name","permit_status"]]

In [66]:
# third, drop duplicate rows in dimension

unique_row = ["permit_type","permittees_business_name","permit_status"]
Permit_dim = Permit_dim.drop_duplicates(subset = unique_row, keep = 'first')
Permit_dim = Permit_dim.reset_index(drop = True)
Permit_dim

,permit_type,permittees_business_name,permit_status
0,NB,LOUIS PERL CONTRACTING LT,ISSUED
1,NB,SUPREME BUILDERS & DEVELO,ISSUED
2,NB,M. MELNICK & COMPANY INC,ISSUED
3,NB,Y&L INTERIOR LLC,ISSUED
4,NB,LEMLE & WOLFF CONSTR CORP,ISSUED
...,...,...,...
2113,NB,M.N.C. GENERAL CONTR CORP,IN PROCESS
2114,NB,M W MANAGEMENT & DEV CORP,RE-ISSUED
2115,NB,ZOITCO CONSTRUCTION CORP,ISSUED
2116,NB,K'S CONSTRUCTION NY INC,ISSUED


In [67]:
# fourth, add permit_id as a surrogate key

Permit_dim.insert(0, 'permit_id', range(1000, 1000 + len(Permit_dim)))
Permit_dim

,permit_id,permit_type,permittees_business_name,permit_status
0,1000,NB,LOUIS PERL CONTRACTING LT,ISSUED
1,1001,NB,SUPREME BUILDERS & DEVELO,ISSUED
2,1002,NB,M. MELNICK & COMPANY INC,ISSUED
3,1003,NB,Y&L INTERIOR LLC,ISSUED
4,1004,NB,LEMLE & WOLFF CONSTR CORP,ISSUED
...,...,...,...,...
2113,3113,NB,M.N.C. GENERAL CONTR CORP,IN PROCESS
2114,3114,NB,M W MANAGEMENT & DEV CORP,RE-ISSUED
2115,3115,NB,ZOITCO CONSTRUCTION CORP,ISSUED
2116,3116,NB,K'S CONSTRUCTION NY INC,ISSUED


In [68]:
#Merge
data = data.merge(Permit_dim,
                  left_on = unique_row,
                  right_on = unique_row,
                  how = 'left')

data.head(10)

,borough,street,zip_code,bldg_type,permit_type,permit_status,permittees_business_name,date,latitude,longitude,year,median_housing_price,new_permit_quantity,location_id,date_id,permit_id
0,BROOKLYN,EAST 22 STREET,11210,1,NB,ISSUED,LOUIS PERL CONTRACTING LT,2019-09-24,40.622307,-73.953736,2019,772000,1,1,20190924,1000
1,BROOKLYN,FLATBUSH AVENUE,11210,2,NB,ISSUED,SUPREME BUILDERS & DEVELO,2019-05-10,40.637919,-73.953055,2019,772000,1,2,20190510,1001
2,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-03-28,40.636955,-73.950995,2019,772000,1,3,20190328,1002
3,BROOKLYN,EAST 32 STREET,11210,2,NB,ISSUED,Y&L INTERIOR LLC,2019-04-30,40.633264,-73.945784,2019,772000,1,4,20190430,1003
4,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,LEMLE & WOLFF CONSTR CORP,2019-01-03,40.636835,-73.945176,2019,772000,1,5,20190103,1004
5,BROOKLYN,ROGERS AVENUE,11210,2,NB,ISSUED,M. MELNICK & COMPANY INC,2019-01-04,40.636955,-73.950995,2019,772000,1,3,20190104,1002
6,BROOKLYN,EAST 32ND ST,11210,2,NB,ISSUED,SUPERB PROPERTY MNG'T,2019-05-07,40.633299,-73.945770,2019,772000,1,6,20190507,1005
7,BROOKLYN,EAST 35TH STREET,11210,2,NB,ISSUED,SINGLA CONCEPTS INC,2019-03-14,40.633586,-73.942913,2019,772000,1,7,20190314,1006
8,BROOKLYN,EAST 31 STREET,11210,1,NB,ISSUED,RELIANT BUILDERS CORP,2019-05-15,40.623316,-73.945209,2019,772000,1,8,20190515,1007
9,BROOKLYN,NEW YORK AVENUE,11210,2,NB,ISSUED,BIG DREAM DEVELOPERS LLC,2019-05-15,40.634770,-73.944933,2019,772000,1,9,20190515,1008


### Step 9: Creating Fact Table

In [69]:
# take a subset of fact_table for only the needed columns: which are keys and measures

New_building_permits_facts = data[['permit_id','date_id','location_id','median_housing_price','new_permit_quantity']]
New_building_permits_facts

,permit_id,date_id,location_id,median_housing_price,new_permit_quantity
0,1000,20190924,1,772000,1
1,1001,20190510,2,772000,1
2,1002,20190328,3,772000,1
3,1003,20190430,4,772000,1
4,1004,20190103,5,772000,1
...,...,...,...,...,...
19217,2630,20210616,4741,450000,1
19218,1418,20210803,7216,465000,1
19219,2036,20210930,7217,465000,1
19220,1418,20211012,7216,465000,1


### Step 10: Load to BigQuery

In [70]:
New_building_permits_facts.to_csv("New_building_permits_facts.csv",index=False)

In [71]:
# create a function to load dataframes to BigQuery

def load_table_to_bigquery(df,
                          table_name,
                          dataset_id):

    dataset_id = dataset_id #change 301800 to match your project id

    dataset_ref = bigquery_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = True
    job_config.write_disposition = "WRITE_TRUNCATE"

    upload_table_name = f"{dataset_id}.{table_name}"
    
    load_job = bigquery_client.load_table_from_dataframe(df,
                                                upload_table_name,
                                                job_config = job_config)
        
    print(f"completed job {load_job}")

In [72]:
#Sending Permit_dim table

load_table_to_bigquery(df = Permit_dim,
                       table_name = "Permit_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=b13fa57a-437d-431e-b26f-0d4282c37a93>


In [73]:
#Sending Date_dim table

load_table_to_bigquery(df = dateDim,
                       table_name = "Date_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=efc08187-a4cf-4c96-a361-baf7136cad27>


In [74]:
#Sending Location_dim table

load_table_to_bigquery(df = Location_dim,
                       table_name = "Location_dim",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=76a2875a-1323-40ad-97fa-c23da0e54fbd>


In [75]:
#Sending Fact table

load_table_to_bigquery(df = New_building_permits_facts,
                       table_name = "New_building_permits_facts",
                          dataset_id = dataset_id)

completed job LoadJob<project=cis-9440-361019, location=US, id=46fd2e21-80f9-4f2b-919d-4816afebfa2b>
